## 감정분석
- 감정 또는 감성(sentiment)
- 텍스트에 나타난 긍정/부정의 태도
- 기계학습 방식과 사전 방식

### 감정사전
- 긍정 표현, 부정 표현의 사전
- 해당 분야 전문가가 있으면 데이터 없이도 만들 수 있음
- 기계학습으로도 개발 가능

## 네이버 영화평

- 네이버 영화에서 분석하고 싶은 영화 페이지로 들어감
- 네티즌 별점 클릭
- 페이지 번호 우클릭 후 주소 복사

In [1]:
url ="https://movie.naver.com/movie/bi/mi/point.nhn?code=101966"

리뷰 수집

In [2]:
import requests
import lxml.html

# 리뷰와 별점을 모을 빈 리스트를 만든다
reviews = []
scores = []

for page in range(1, 30):  # 1~29페이지까지 반복
    res = requests.get(url.format(page))  # 각 페이지에 접속한다
    root = lxml.html.fromstring(res.text)  # html을 처리한다

    # 리뷰를 가져와 reviews에 추가한다
    for review in root.cssselect('.score_reple p'):
        reviews.append(review.text_content())

    # 별점을 가져와 scores에 추가한다
    for score in root.cssselect('.score_result .star_score em'):
        scores.append(int(score.text_content()))

C:\ProgramData\Anaconda3\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.3) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


표 만들기

In [3]:
import pandas as pd

df = pd.DataFrame({
    'score': scores, 
    'review': reviews
})

In [4]:
df.head()

,score,review
0,7,불씨를 횃불로 키우는 재능
1,8,시리즈 전체가 하나의 True American Classic
2,9,"관계와 구속, 존재 이유에 대한 시리즈의 새로운 질문"
3,9,"1편의 정통이자, 3편의 감동을 뛰어넘는다"
4,7,‘내 아이’를 넘어서는 발상의 전환. 토이 유니버스의 시작?


긍/부정 표시

In [5]:
import numpy as np

In [6]:
df['sentiment'] = np.where(df['score'] > 5, 1, 0)
df.head()

,score,review,sentiment
0,7,불씨를 횃불로 키우는 재능,1
1,8,시리즈 전체가 하나의 True American Classic,1
2,9,"관계와 구속, 존재 이유에 대한 시리즈의 새로운 질문",1
3,9,"1편의 정통이자, 3편의 감동을 뛰어넘는다",1
4,7,‘내 아이’를 넘어서는 발상의 전환. 토이 유니버스의 시작?,1


In [7]:
# 저장
df.to_csv('movie_review.csv', encoding='utf8', index=False)

In [8]:
# 불러오기
df = pd.read_csv('movie_review.csv', encoding='utf8')

한글, 알파벳, 숫자 제외한 문자 제거

In [9]:
import re

In [10]:
def remove_non_word(text):
    """한글, 알파벳, 숫자를 제외한 문자를 제거"""
    return re.sub(r'[^가-힣A-z0-9]+', ' ', text)

In [11]:
remove_non_word('Wow, 정말 1도 재미 없다!')

'Wow 정말 1도 재미 없다 '

WPM(Word Piece Model) 학습
 - 구글의 WPM에는 BPE(Byte Pair Encoding) 알고리즘
    - 자연어 처리를 위한 주요 전처리 방법으로 사용

In [12]:
from subword_nmt.learn_bpe import learn_bpe
import io

with open('영화평BPE.txt', 'w', encoding='utf8') as outfile:
    infile = io.StringIO(remove_non_word(' '.join(df['review'])))
    learn_bpe(infile, outfile, 1000)

no pair has frequency >= 2. Stopping


In [13]:
from subword_nmt.apply_bpe import BPE

with open('영화평BPE.txt', encoding='utf8') as f:
    bpe = BPE(f, separator='~')

TDM 만들기

- BoW(Bag of Words)란 단어들의 순서는 전혀 고려하지 않고, 단어들의 출현 빈도(frequency)에만 집중하는 텍스트 데이터의 수치화 표현 방법
<img src="TDM.jpg">

In [14]:
def tokenizer_wpm(text):
    text = remove_non_word(text)
    tokens = bpe.process_line(text)
    tokens = tokens.split()
    return [t for t in tokens
            if (not t.endswith('~') and len(t) > 1) or len(t) > 2]

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

In [16]:
cv_wpm = CountVectorizer(max_features=1000, tokenizer=tokenizer_wpm)

In [17]:
tdm = cv_wpm.fit_transform(df['review'])

토큰 빈도

In [18]:
freq = pd.DataFrame({
    'word': cv_wpm.get_feature_names(),
    'n': tdm.sum(axis=0).flat
})

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\dtypes\dtypes.py:458: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if string == 'category':


In [28]:
freq.sort_values('n').head(10)

,word,n
0,1편의,29
20,시작,29
21,아이,29
22,우정의,29
23,유니버스의,29
24,이유에,29
25,이제는,29
26,재능,29
27,전체가,29
28,전환,29


In [38]:
from bs4 import BeautifulSoup
import requests
from collections import Counter
from WordCloud import WordCloud
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'WordCloud'

In [ ]:
#!conda install -c conda-forge wordcloud

In [39]:
pip install wordcloud

Note: you may need to restart the kernel to use updated packages.


In [40]:
from wordcloud import WordCloud

In [44]:
#wc = WordCloud(width=400, height=400, background_color='white')
wc = WordCloud(font_path='C:\Windows\Fonts\malgun.ttf', width=400, height=400, background_color='white')
# 한글 파일일 경우 폰트 지정할 것.

In [46]:

frequencies = {word: freq for word, freq in zip(freq.get_feature_names(), tdm.flat)}
# dictionery 형태로 변경

AttributeError: 'DataFrame' object has no attribute 'get_feature_names'

In [45]:
wc.fit_words(frequencies)

NameError: name 'frequencies' is not defined

데이터 분할

In [20]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(
    tdm, df['sentiment'], test_size=.2, random_state=1234)

학습

In [21]:
from sklearn.linear_model import LogisticRegressionCV
model = LogisticRegressionCV(random_state=1234)
model.fit(x_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 1

성능 평가

In [23]:
from sklearn.metrics import accuracy_score
y_pred = model.predict(x_test)
accuracy_score(y_test, y_pred)

NotFittedError: This LogisticRegressionCV instance is not fitted yet

계수 분석

In [24]:
word_coef = pd.DataFrame({
    'word': cv_wpm.get_feature_names(),
    'coef': model.coef_.flat
})

AttributeError: 'LogisticRegressionCV' object has no attribute 'coef_'

In [25]:
word_coef.head()

NameError: name 'word_coef' is not defined

In [26]:
word_coef.sort_values('coef', ascending=False).head(10)

NameError: name 'word_coef' is not defined

In [27]:
word_coef.sort_values('coef').tail(10)

NameError: name 'word_coef' is not defined